In [1]:
from sapiopylib.rest.User import SapioUser
from sapiopylib.rest.DataMgmtService import DataMgmtServer
from sapiopylib.rest.pojo.CustomReport import *
from IPython.display import display, HTML
import pandas as pd
pd.set_option('display.max_columns', 100)

user = SapioUser(url="https://linux-vm:8443/webservice/api",
                 guid="3c232543-f407-4828-aae5-b33d4cd31fa7", account_name="sapio",
                 username="pyRestTest", password="Password1!", verify_ssl_cert=False)
customReportManager = DataMgmtServer.get_custom_report_manager(user)

Custom Report API offers a way for users to send complex queries about data records across a hierarchy.

In the first example, we will run a quick report, which is consisted with only a single term about a data record.
We will find all samples with sample type 'Blood' in the system, but only for the first 100 samples in the system.

We can obtain the data frame from report result so it can be organized as pandas data frame and display it.

In [2]:
root_term = RawReportTerm('Sample', 'ExemplarSampleType', RawTermOperation.EQUAL_TO_OPERATOR, 'Blood', is_negated=True)
report = customReportManager.run_quick_report(root_term, page_size=100)
data_frame = report.get_data_frame()
display(data_frame)

/home/yqiao/anaconda3/envs/RDKit/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'linux-vm'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


,Sample.ObservedCondition,Sample.MultiParentLink234,Sample.OtherSampleId,Sample.ActiveWorkflowId,Sample.RelatedNotebookExperiment,Sample.SacrificedDate,Sample.RelatedRecord170,Sample.SampleId,Sample.StorageLocationBarcode,Sample.Organism,Sample.CreatedBy,Sample.CollectionDateTime,Sample.RelatedRecord146,Sample.ElnbNumber,Sample.PercentRecoveries,Sample.DataRecordName,Sample.Volume,Sample.ExemplarSampleType,Sample.RelatedChild123,Sample.ControlType,Sample.RelatedRecord181,Sample.TotalMass,Sample.ConcentrationUnits,Sample.RelatedRecord180,Sample.CollectionDate,Sample.RelatedRecord112,Sample.RelatedRecord111,Sample.RelatedRecord199,Sample.RelatedRecord110,Sample.DateCreated,Sample.MultiParentLink176,Sample.RelatedRecord119,Sample.RowPosition,Sample.ColPosition,Sample.VeloxLastModifiedBy,Sample.RecordId,Sample.ExemplarSampleStatus,Sample.MultiParentLink247,Sample.VolumeUnits,Sample.RelatedRecord168,Sample.RelatedRecord200,Sample.Comments,Sample.ContainerType,Sample.PreviousExemplarStatus,Sample.TimePoint,Sample.Concentration,Sample.Flags,Sample.RelatedPlate,Sample.MultiParentLink243,Sample.RelatedRecord208,Sample.StorageUnitPath,Sample.SapioSingleTemplateProcess,Sample.LastEditedBy,Sample.VeloxLastModifiedDate,Sample.PlateId,Sample.RelatedChild149,Sample.IsInvalid
0,,None,,None,,None,,000004,,,sdmsapi,None,,,None,000004,None,,None,,,None,ng/uL,,None,,,,,1663688244010,None,,,,sdmsapi,130,,None,,,,,,,None,None,,,None,,,,,1663688244522,,None,False
1,,None,,None,,None,,000005,,,sdmsapi,None,,,None,000005,None,,None,,,None,ng/uL,,None,,,,,1663688248670,None,,,,sdmsapi,132,,None,,,,,,,None,None,,,None,,,,,1663688248737,,None,False
2,,None,,None,,None,,000006,,,sdmsapi,None,,,None,000006,None,Flow Cytometry,None,,,None,ng/uL,,None,,,,,1663688707312,None,,,,sdmsapi,169,,None,,,,,,,None,None,,,None,,,,,1663688707708,,None,False
3,,None,export_COVID19 samples 23_04_20_ST3_COVID19_IC...,None,,None,,000007,,,sdmsapi,None,,,None,000007 export_COVID19 samples 23_04_20_ST3_COV...,None,Flow Cytometry,None,,,None,ng/uL,,None,,,,,1663693730249,None,,,,sdmsapi,171,,None,,,,,,,None,None,,,None,,,,,1663693730582,,None,False
4,,None,,None,,None,,PyWebDemo-13,,,pyRestTest,None,,,None,PyWebDemo-13,None,,None,,,None,ng/uL,,None,,,,,1663881353164,None,,,,pyRestTest,219,,None,,,7,,,,None,None,,,None,,,,,1663881353236,,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,,None,,None,,None,,PyWebDemo-102,,,pyRestTest,None,,,None,PyWebDemo-102,None,,None,,,None,ng/uL,,None,,,,,1665088557296,None,,,,pyRestTest,438,,None,,,,,,,None,None,,,None,,,,,1665088557759,,None,False
84,,None,,None,,None,,PyWebDemo-103,,,pyRestTest,None,,,None,PyWebDemo-103,None,,None,,,None,ng/uL,,None,,,,,1665088557297,None,,,,pyRestTest,439,,None,,,,,,,None,None,,,None,,,,,1665088557759,,None,False
85,,None,,None,,None,,PyWebDemo-104,,,pyRestTest,None,,,None,PyWebDemo-104,None,,None,,,None,ng/uL,,None,,,,,1665088557297,None,,,,pyRestTest,440,,None,,,,,,,None,None,,,None,,,,,1665088557759,,None,False
86,,None,,None,,None,,PyWebDemo-105,,,pyRestTest,None,,,None,PyWebDemo-105,None,,None,,,None,ng/uL,,None,,,,,1665088557297,None,,,,pyRestTest,441,,None,,,,,,,None,None,,,None,,,,,1665088557759,,None,False


In a more advanced report, you may want to enter multiple conditions, across a hierarchy of records.
Here's a way to get all attachments under a location '205 N George St'

In [3]:
term1 = RawReportTerm('VeloxLocation', 'LocationName', RawTermOperation.EQUAL_TO_OPERATOR, '205 N George St')
term2 = RawReportTerm('Attachment', 'RecordId', RawTermOperation.GREATER_THAN_OR_EQUAL_OPERATOR, '0')
root_term = CompositeReportTerm(term1, CompositeTermOperation.AND_OPERATOR, term2)
column_list = [ReportColumn('Attachment', 'FilePath', FieldType.STRING), ReportColumn('Attachment', 'RecordId', FieldType.LONG)]
request = CustomReportCriteria(column_list, root_term, page_size=1)
report = customReportManager.run_custom_report(request)
data_frame = report.get_data_frame()
display(data_frame)

/home/yqiao/anaconda3/envs/RDKit/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'linux-vm'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


,Attachment.FilePath,Attachment.RecordId
0,2360814_166058953148747601.pdf,111
1,Est.pdf,112
2,Sapio Mutual NDA.docx,116


In Sapio 22.10, we can add **explicit joins** to records of types that are **not hierarchically related**.

In this case, let's find the sample's receipt status along with the occupancy info of the storage unit the sample "00001" is under.

Observe that in *ExplicitJoinDefinition*, the data type name outside of the join term must be set to the **Unrelated Data Type Name** in the report. In this case, storage units are not related to either sample or sample receipt via a hierarchy.

In [5]:
sample_term = RawReportTerm('Sample', 'SampleId', RawTermOperation.EQUAL_TO_OPERATOR, '00001')
receipt_term = RawReportTerm('SampleReceipt', 'RecordId', RawTermOperation.GREATER_THAN_OPERATOR, '0')
root_term = CompositeReportTerm(sample_term, CompositeTermOperation.AND_OPERATOR, receipt_term)
storage_join_term = FieldCompareReportTerm('Sample', 'StorageLocationBarcode', RawTermOperation.EQUAL_TO_OPERATOR, 'StorageUnit', 'StorageUnitId')
column_list = [ReportColumn('Sample', 'SampleId', FieldType.STRING), ReportColumn('SampleReceipt', 'SampleReceivedRejected', FieldType.PICKLIST),
               ReportColumn('StorageUnit', 'OccupiedCount', FieldType.LONG), ReportColumn('StorageUnit', 'StorageUnitCapacity', FieldType.LONG)]
request = CustomReportCriteria(column_list=column_list, root_term=root_term, join_list=[ExplicitJoinDefinition('StorageUnit', storage_join_term)])
report = customReportManager.run_custom_report(request)
data_frame = report.get_data_frame()
display(data_frame)

/home/yqiao/anaconda3/envs/RDKit/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'linux-vm'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


,Sample.SampleId,SampleReceipt.SampleReceivedRejected,StorageUnit.OccupiedCount,StorageUnit.StorageUnitCapacity
0,00001,Received,1,16
